In [6]:
#call_1 = `source activate sentinel`
#call_2 = `gdalwarp -overwrite /Users/simon/Data/DEM/srtm_39_01/srtm_39_01.tif /Users/simon/Data/DEM/srtm_39_01/srtm_39_01_reproject.tif -s_srs EPSG:4326+5773 -t_srs EPSG:4979`

`gdalwarp -overwrite /Users/simon/Data/DEM/srtm_39_01/srtm_39_01.tif /Users/simon/Data/DEM/srtm_39_01/srtm_39_01_reproject.tif -s_srs EPSG:4326+5773 -t_srs EPSG:4979`

In [9]:
import PyCall

In [10]:
rasterio = PyCall.pyimport("rasterio")
dem39 = rasterio.open("/Users/simon/Data/DEM/srtm_39_01/srtm_39_01_reproject.tif")
dem38 = rasterio.open("/Users/simon/Data/DEM/srtm_38_01/srtm_38_01_transformed.tif")

PyObject <open DatasetReader name='/Users/simon/Data/DEM/srtm_38_01/srtm_38_01_transformed.tif' mode='r'>

In [11]:
dem38.meta

Dict{Any,Any} with 8 entries:
  "driver"    => "GTiff"
  "dtype"     => "int16"
  "height"    => 6000
  "nodata"    => -32768.0
  "count"     => 1
  "width"     => 6000
  "transform" => (0.000833333, 0.0, 5.0, 0.0, -0.000833333, 60.0, 0.0, 0.0, 1.0)
  "crs"       => PyObject CRS.from_epsg(4979)

In [12]:
dem39.meta

Dict{Any,Any} with 8 entries:
  "driver"    => "GTiff"
  "dtype"     => "int16"
  "height"    => 6000
  "nodata"    => -32768.0
  "count"     => 1
  "width"     => 6000
  "transform" => (0.000833333, 0.0, 10.0, 0.0, -0.000833333, 60.0, 0.0, 0.0, 1.…
  "crs"       => PyObject CRS.from_epsg(4979)

In [16]:
transform = dem38.get_transform()
rows = collect(1:dem38.height).-1;
columns = collect(1:dem38.width).-1;
lon38 = transform[1] .+ rows .* transform[2];
lat38  = transform[4] .+ columns .* transform[6];

In [20]:
lat38.-lat39

6000-element Array{Float64,1}:
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 ⋮                    
 0.0                  
 0.0                  
 7.105427357601002e-15
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.0                  

In [19]:
transform = dem39.get_transform()
rows = collect(1:dem39.height).-1;
columns = collect(1:dem39.width).-1;
lon39 = transform[1] .+ rows .* transform[2];
lat39  = transform[4] .+ columns .* transform[6];

In [23]:
lon38

6000-element Array{Float64,1}:
 4.999999999999999 
 5.000833333333333 
 5.001666666666666 
 5.0024999999999995
 5.003333333333332 
 5.0041666666666655
 5.004999999999999 
 5.0058333333333325
 5.006666666666666 
 5.007499999999999 
 5.008333333333333 
 5.009166666666665 
 5.009999999999999 
 ⋮                 
 9.989999999999995 
 9.99083333333333  
 9.991666666666662 
 9.992499999999996 
 9.993333333333329 
 9.994166666666661 
 9.994999999999996 
 9.99583333333333  
 9.996666666666663 
 9.997499999999995 
 9.998333333333328 
 9.999166666666662 

In [22]:
lon39

6000-element Array{Float64,1}:
  9.999999999999998
 10.00083333333333 
 10.001666666666665
 10.002499999999998
 10.003333333333332
 10.004166666666665
 10.004999999999999
 10.005833333333332
 10.006666666666664
 10.007499999999999
 10.008333333333331
 10.009166666666665
 10.009999999999998
  ⋮                
 14.989999999999995
 14.990833333333327
 14.991666666666662
 14.992499999999996
 14.993333333333329
 14.994166666666661
 14.994999999999996
 14.995833333333328
 14.996666666666663
 14.997499999999995
 14.998333333333328
 14.999166666666662

In [39]:
lon = cat(lon38,lon39,dims=1)
heights = hcat(dem38.read(1),dem39.read(1))

6000×12000 Array{Int16,2}:
 -32768  -32768  -32768  -32768  -32768  …  309  308  308  308  307  306  304
 -32768  -32768  -32768  -32768  -32768     312  311  310  308  306  304  302
 -32768  -32768  -32768  -32768  -32768     310  311  311  310  306  301  299
 -32768  -32768  -32768  -32768  -32768     309  308  308  308  306  304  302
 -32768  -32768  -32768  -32768  -32768     309  311  311  312  313  315  316
 -32768  -32768  -32768  -32768  -32768  …  311  312  316  320  321  323  325
 -32768  -32768  -32768  -32768  -32768     315  316  319  322  325  328  329
 -32768  -32768  -32768  -32768  -32768     317  316  319  323  328  333  332
 -32768  -32768  -32768  -32768  -32768     320  319  320  321  326  331  330
 -32768  -32768  -32768  -32768  -32768     314  315  317  319  323  327  326
 -32768  -32768  -32768  -32768  -32768  …  308  309  310  312  316  320  319
 -32768  -32768  -32768  -32768  -32768     301  302  303  304  307  310  311
 -32768  -32768  -32768  -32768  -327

In [40]:
import JLD

In [43]:
JLD.save("/Users/simon/Data/DEM/denmark.jld", "lat", lat38, "lon", lon, "height", heights)

In [28]:
import Colors

In [36]:
test = heights;
test = convert.(Float64,test)
test[test .<0] .= 0.1 
test[test .>0.5] .= 0.9;

In [ ]:
Colors.Gray.(test[1:10:end,1:10:end])